# Point Source Modeling with PyAERMOD

This notebook provides detailed examples of point source modeling using PyAERMOD.

## Topics Covered

1. Single stack with varying parameters
2. Multiple stacks at a facility
3. Stack parameter optimization
4. Source groups and urban sources
5. Elevated and ground-level releases

**Prerequisites:**
- PyAERMOD installed
- Basic understanding of AERMOD concepts
- Meteorological data files (optional for some examples)

## Setup

In [ ]:
# Import PyAERMOD
from pyaermod.input_generator import (
    AERMODProject,
    ControlPathway,
    SourcePathway,
    PointSource,
    ReceptorPathway,
    CartesianGrid,
    PolarGrid,
    DiscreteReceptor,
    MeteorologyPathway,
    OutputPathway,
    PollutantType,
    TerrainType
)

import numpy as np
import pandas as pd

print("✓ PyAERMOD modules imported successfully")

## Example 1: Basic Stack with Varying Exit Velocity

Understanding how exit velocity affects plume rise and dispersion.

In [ ]:
# Test three exit velocities: low, medium, high
exit_velocities = [5.0, 15.0, 25.0]  # m/s

for vel in exit_velocities:
    control = ControlPathway(
        title_one=f"Exit Velocity Study: {vel} m/s",
        title_two="Single stack with varying exit velocity",
        pollutant_id=PollutantType.PM25,
        averaging_periods=["ANNUAL"],
        terrain_type=TerrainType.FLAT
    )
    
    sources = SourcePathway()
    sources.add_source(PointSource(
        source_id="STACK1",
        x_coord=0.0,
        y_coord=0.0,
        base_elevation=100.0,
        stack_height=50.0,
        stack_temp=400.0,  # K
        exit_velocity=vel,
        stack_diameter=2.0,
        emission_rate=1.0  # g/s
    ))
    
    # Polar grid centered on stack
    receptors = ReceptorPathway()
    receptors.add_polar_grid(PolarGrid(
        x_origin=0.0,
        y_origin=0.0,
        dist_init=100.0,   # first ring at 100m
        dist_num=6,         # 6 rings
        dist_delta=300.0,   # 300m spacing (100, 400, 700, 1000, 1300, 1600m)
        dir_init=0.0,       # start at north
        dir_num=36,         # 36 directions
        dir_delta=10.0,     # every 10 degrees
    ))
    
    meteorology = MeteorologyPathway(
        surface_file="met.sfc",
        profile_file="met.pfl"
    )
    
    output = OutputPathway(
        receptor_table=True,
        max_table=True
    )
    
    project = AERMODProject(
        control=control,
        sources=sources,
        receptors=receptors,
        meteorology=meteorology,
        output=output
    )
    
    filename = f"point_source_vel_{vel:.0f}ms.inp"
    project.write(filename)
    print(f"✓ Created: {filename}")

print("\n✓ Exit velocity study files created")
print("\nKey insight: Higher exit velocity → Better plume rise → Lower ground-level concentrations")

## Example 2: Multiple Stacks at Industrial Facility

Model a facility with several emission points.

In [ ]:
control = ControlPathway(
    title_one="Multi-Stack Industrial Facility",
    title_two="3 stacks with different characteristics",
    pollutant_id=PollutantType.NOX,
    averaging_periods=["ANNUAL", "24", "1"],
    terrain_type=TerrainType.FLAT
)

sources = SourcePathway()

# Main boiler stack (tall, hot, high velocity)
sources.add_source(PointSource(
    source_id="BOILER",
    x_coord=0.0,
    y_coord=0.0,
    base_elevation=100.0,
    stack_height=75.0,  # 75m tall
    stack_temp=450.0,   # Hot exhaust
    exit_velocity=20.0,
    stack_diameter=2.5,
    emission_rate=5.0,  # g/s - highest emission
    source_groups=["ALL", "COMBUSTION"]
))

# Process vent (shorter, cooler)
sources.add_source(PointSource(
    source_id="PROCESS1",
    x_coord=50.0,
    y_coord=30.0,
    base_elevation=100.0,
    stack_height=25.0,
    stack_temp=330.0,
    exit_velocity=8.0,
    stack_diameter=1.0,
    emission_rate=1.5,  # g/s
    source_groups=["ALL", "PROCESS"]
))

# Emergency generator (short, intermittent)
sources.add_source(PointSource(
    source_id="GENERATOR",
    x_coord=-40.0,
    y_coord=-20.0,
    base_elevation=100.0,
    stack_height=15.0,
    stack_temp=400.0,
    exit_velocity=12.0,
    stack_diameter=0.5,
    emission_rate=0.8,  # g/s
    source_groups=["ALL", "BACKUP"]
))

print(f"✓ Defined {len(sources.sources)} sources:")
for src in sources.sources:
    print(f"  - {src.source_id}: h={src.stack_height}m, Q={src.emission_rate} g/s")

# Cartesian grid covering facility and surroundings
receptors = ReceptorPathway()
receptors.add_cartesian_grid(
    CartesianGrid.from_bounds(
        x_min=-1000,
        x_max=1000,
        y_min=-1000,
        y_max=1000,
        spacing=50  # 50m resolution
    )
)

# Add discrete receptors at property boundary
boundary_receptors = [
    ("FENCE_N", 0, 200),
    ("FENCE_S", 0, -200),
    ("FENCE_E", 200, 0),
    ("FENCE_W", -200, 0),
]

for rec_id, x, y in boundary_receptors:
    receptors.add_discrete_receptor(
        DiscreteReceptor(x_coord=x, y_coord=y, z_elev=100.0)
    )

print(f"\n✓ Defined receptors: Grid + {len(boundary_receptors)} fence line points")

meteorology = MeteorologyPathway(
    surface_file="met.sfc",
    profile_file="met.pfl"
)

output = OutputPathway(
    receptor_table=True,
    receptor_table_rank=50,  # Top 50 receptors
    max_table=True,
    summary_file="multi_stack.sum"
)

project = AERMODProject(
    control=control,
    sources=sources,
    receptors=receptors,
    meteorology=meteorology,
    output=output
)

project.write("multi_stack_facility.inp")
print("\n✓ Created: multi_stack_facility.inp")

## Example 3: Stack Height Optimization

Find the minimum stack height needed to meet air quality standards.

In [ ]:
# Test stack heights from 20m to 80m
stack_heights = list(range(20, 85, 10))  # [20, 30, 40, 50, 60, 70, 80]

print("Stack Height Optimization Study")
print("="*70)
print("Testing heights:", stack_heights)
print()

for height in stack_heights:
    control = ControlPathway(
        title_one=f"Stack Height Optimization: {height}m",
        title_two="Finding minimum height for compliance",
        pollutant_id=PollutantType.SO2,
        averaging_periods=["ANNUAL", "24", "1"],
        terrain_type=TerrainType.FLAT
    )
    
    sources = SourcePathway()
    sources.add_source(PointSource(
        source_id="STACK1",
        x_coord=0.0,
        y_coord=0.0,
        base_elevation=100.0,
        stack_height=float(height),
        stack_temp=420.0,
        exit_velocity=15.0,
        stack_diameter=2.0,
        emission_rate=2.5  # g/s
    ))
    
    # Dense grid near source
    receptors = ReceptorPathway()
    receptors.add_polar_grid(PolarGrid(
        x_origin=0.0,
        y_origin=0.0,
        distances=list(range(100, 3100, 100)),  # 100m to 3km, every 100m
        angles=list(range(0, 360, 5))  # Every 5 degrees
    ))
    
    meteorology = MeteorologyPathway(
        surface_file="met.sfc",
        profile_file="met.pfl"
    )
    
    output = OutputPathway(
        receptor_table=True,
        max_table=True
    )
    
    project = AERMODProject(
        control=control,
        sources=sources,
        receptors=receptors,
        meteorology=meteorology,
        output=output
    )
    
    filename = f"stack_optimization_h{height}m.inp"
    project.write(filename)
    print(f"✓ Height={height}m: {filename}")

print("\n✓ Stack height optimization study complete")
print("\nNext steps:")
print("1. Run all scenarios with AERMOD")
print("2. Parse outputs and extract maximum concentrations")
print("3. Plot max concentration vs. stack height")
print("4. Identify minimum height meeting standards")

## Example 4: Urban vs. Rural Sources

Compare dispersion in urban and rural settings.

In [ ]:
# Create two scenarios: rural and urban
scenarios = [
    {"name": "rural", "is_urban": False, "terrain": TerrainType.FLAT},
    {"name": "urban", "is_urban": True, "terrain": TerrainType.FLAT}
]

for scenario in scenarios:
    control = ControlPathway(
        title_one=f"Urban/Rural Comparison: {scenario['name'].upper()}",
        title_two="Same source, different environment",
        pollutant_id=PollutantType.CO,
        averaging_periods=["1", "8"],
        terrain_type=scenario['terrain']
    )
    
    sources = SourcePathway()
    
    # Urban designation affects dispersion coefficients
    stack = PointSource(
        source_id="STACK1",
        x_coord=0.0,
        y_coord=0.0,
        base_elevation=100.0,
        stack_height=40.0,
        stack_temp=380.0,
        exit_velocity=12.0,
        stack_diameter=1.5,
        emission_rate=10.0,  # g/s
        is_urban=scenario['is_urban']
    )
    
    if scenario['is_urban']:
        stack.urban_area_name = "CITYAREA"
    
    sources.add_source(stack)
    
    receptors = ReceptorPathway()
    receptors.add_cartesian_grid(
        CartesianGrid.from_bounds(
            x_min=-2000,
            x_max=2000,
            y_min=-2000,
            y_max=2000,
            spacing=100
        )
    )
    
    meteorology = MeteorologyPathway(
        surface_file="met.sfc",
        profile_file="met.pfl"
    )
    
    output = OutputPathway(
        receptor_table=True,
        max_table=True
    )
    
    project = AERMODProject(
        control=control,
        sources=sources,
        receptors=receptors,
        meteorology=meteorology,
        output=output
    )
    
    filename = f"urban_rural_comparison_{scenario['name']}.inp"
    project.write(filename)
    print(f"✓ Created: {filename} (is_urban={scenario['is_urban']})")

print("\n✓ Urban/rural comparison files created")
print("\nKey differences:")
print("- Urban: Enhanced vertical mixing, higher surface roughness")
print("- Rural: Less mixing, smoother surface")
print("- Urban typically has lower peak concentrations but wider dispersion")

## Example 5: Ground-Level vs. Elevated Release

Compare emissions from ground level (e.g., loading dock) vs. elevated (stack).

In [ ]:
# Ground-level release (loading dock fugitive emissions)
control_ground = ControlPathway(
    title_one="Ground-Level Release",
    title_two="Loading dock fugitive emissions",
    pollutant_id=PollutantType.PM10,
    averaging_periods=["24", "ANNUAL"],
    terrain_type=TerrainType.FLAT
)

sources_ground = SourcePathway()
sources_ground.add_source(PointSource(
    source_id="LOADING",
    x_coord=0.0,
    y_coord=0.0,
    base_elevation=100.0,
    stack_height=3.0,  # Very short - essentially ground level
    stack_temp=288.0,  # Ambient temperature (15°C)
    exit_velocity=0.1,  # Minimal velocity
    stack_diameter=2.0,
    emission_rate=0.5  # g/s
))

# Same emission rate but elevated
control_elevated = ControlPathway(
    title_one="Elevated Release",
    title_two="Stack emissions",
    pollutant_id=PollutantType.PM10,
    averaging_periods=["24", "ANNUAL"],
    terrain_type=TerrainType.FLAT
)

sources_elevated = SourcePathway()
sources_elevated.add_source(PointSource(
    source_id="STACK",
    x_coord=0.0,
    y_coord=0.0,
    base_elevation=100.0,
    stack_height=30.0,  # 30m stack
    stack_temp=350.0,
    exit_velocity=10.0,
    stack_diameter=1.0,
    emission_rate=0.5  # Same emission rate
))

# Common receptor grid
receptors = ReceptorPathway()
receptors.add_cartesian_grid(
    CartesianGrid.from_bounds(
        x_min=-500,
        x_max=500,
        y_min=-500,
        y_max=500,
        spacing=25  # Fine resolution
    )
)

meteorology = MeteorologyPathway(
    surface_file="met.sfc",
    profile_file="met.pfl"
)

output = OutputPathway(
    receptor_table=True,
    max_table=True
)

# Create both projects
for name, ctrl, srcs in [("ground", control_ground, sources_ground),
                          ("elevated", control_elevated, sources_elevated)]:
    project = AERMODProject(
        control=ctrl,
        sources=srcs,
        receptors=receptors,
        meteorology=meteorology,
        output=output
    )
    
    filename = f"release_height_{name}.inp"
    project.write(filename)
    print(f"✓ Created: {filename}")

print("\n✓ Ground vs. elevated release comparison complete")
print("\nExpected results:")
print("- Ground-level: Higher peak concentrations close to source")
print("- Elevated: Lower near-field concentrations, better dilution")
print("- This demonstrates the importance of stack height for air quality!")

## Summary

This notebook demonstrated:

1. **Exit velocity effects** - Higher velocity improves plume rise
2. **Multi-stack facilities** - Modeling complex industrial sites
3. **Stack height optimization** - Finding minimum height for compliance
4. **Urban vs. rural** - Environmental setting affects dispersion
5. **Release height comparison** - Elevation dramatically reduces impacts

### Key Takeaways

- Point sources are characterized by: location, height, temperature, velocity, diameter, emission rate
- Stack height is often the most effective control for ground-level concentrations
- Urban designation accounts for enhanced mixing in cities
- Source groups allow analyzing subsets of emissions
- Multiple sources can interact through plume overlap

### Next Steps

- **Notebook 03**: Area Source Modeling
- **Notebook 04**: Parameter Sweeps and Batch Processing
- **Notebook 05**: Visualization Techniques